In [1]:
import pandas as pd
pv = pd.read_csv("../data/pv_cf.csv", parse_dates=["timesteps"])
dm = pd.read_csv("../data/demand_h2_MWh.csv", parse_dates=["timesteps"])

print("pv:", pv.timesteps.min(), pv.timesteps.max(), len(pv), "dups:", pv.timesteps.duplicated().sum())
print("dm:", dm.timesteps.min(), dm.timesteps.max(), len(dm), "dups:", dm.timesteps.duplicated().sum())

# ¿coinciden exactamente?
print("índices iguales:", pv.timesteps.equals(dm.timesteps))

# Muestra primeras diferencias
a = pv.timesteps[~pv.timesteps.isin(dm.timesteps)]
b = dm.timesteps[~dm.timesteps.isin(pv.timesteps)]
print("en pv y no en demanda:", a.head(3).to_list())
print("en demanda y no en pv:", b.head(3).to_list())

pv: 2023-01-01 00:00:00 2043-12-31 23:00:00 184080 dups: 0
dm: 2023-01-01 00:00:00 2043-12-31 23:00:00 184080 dups: 0
índices iguales: True
en pv y no en demanda: []
en demanda y no en pv: []


In [2]:
dm = dm.drop_duplicates(subset=["timesteps"], keep="first")

# (Si hubiera mismos timestamps con valores distintos, usar agregación)
# dm = dm.groupby("timesteps", as_index=False).agg({"VALPO.demand_h2":"mean"})

# 2) Alinear exactamente al índice de PV (por si acaso)
dm = dm.set_index("timesteps").reindex(pv["timesteps"]).rename_axis("timesteps").reset_index()

# 3) Verificación y guardar
assert pv["timesteps"].equals(dm["timesteps"])
dm.to_csv("../data/demand_h2_MWh.csv", index=False)
print("OK: demanda sin duplicados y alineada")


OK: demanda sin duplicados y alineada
